In [2]:
import ray
'''
SAMPLE CODE TO VALIDATE RUNTIME_ENV
'''
runtime_env = {
    "eager_install":True,
    "pip": ["imblearn","minio", "urllib3", "requests", "pandas", "scipy", "kfp"], 
    "env_vars":{"HTTP_PROXY": "http://10.78.90.46:80", "HTTPS_PROXY": "http://10.78.90.46:80", "http_proxy": "http://10.78.90.46:80", "https_proxy": "http://10.78.90.46:80"},
    "pip_check": False
}

ray.init(
    address="ray://kuberay-head-svc.kuberay:10001", 
    ignore_reinit_error=True, 
    runtime_env=runtime_env, 
    _temp_dir="/tmp/ray/", 
    local_mode=False,
    namespace="kuberay")

print("check1")
@ray.remote(runtime_env=runtime_env)
# @ray.remote
class A:
    def __call__(self, *args, **kwargs):
        return "hello Avi, welcome to Ray !"
    
    def overspaling_smote(self):
        import json
        from minio import Minio
        from imblearn.over_sampling import SMOTE
        import urllib3
        import uuid
        import requests
        import pandas as pd
        from requests.packages.urllib3.exceptions import InsecureRequestWarning
        requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

        configuration_dtl = {
            "MINIO_HOST_URL":"home.hpe-apps-ezaf.com:31900",
            "MINIO_ACCESS_KEY":"minioadmin",
            "MINIO_SECRET_KEY":"minioadmin",
            "SOURCE_PATH":"/source/feed-2.csv",
            "GENERATED_PATH":"/source/generated-data.csv",
            "storage_uri": "s3://experiments/ray/pickels/logisticregression/model", 
            "protocol_version": "v2",
            "bucket_name": "experiments",
            "EZAF_ENV":"hpe-staging-ezaf",
            "token_url":"https://keycloak.{0}.com/realms/UA/protocol/openid-connect/token"
        }

        print("configuration_dtl :: ===== ::", json.loads(json.dumps(configuration_dtl)))
        MINIO_CLIENT_INFR = Minio(
            endpoint= configuration_dtl.get('MINIO_HOST_URL'), 
            access_key=configuration_dtl.get('MINIO_ACCESS_KEY'), 
            secret_key=configuration_dtl.get('MINIO_SECRET_KEY'),
            secure=True,
            http_client = urllib3.PoolManager(cert_reqs='CERT_NONE'))

        print("MINIO_CLIENT", MINIO_CLIENT_INFR)
        csv_file = MINIO_CLIENT_INFR.get_object(configuration_dtl.get('bucket_name'), configuration_dtl.get('GENERATED_PATH'))
        data = pd.read_csv(csv_file)
        data.head(5)
        data_reduced = data.drop(['zipcodeOri','zipMerchant'],axis=1)
        data_reduced.loc[:,['customer','merchant','category']].astype('category')

        # turning object columns type to categorical for easing the transformation process
        col_categorical = data_reduced.select_dtypes(include= ['object']).columns
        for col in col_categorical:
            data_reduced[col] = data_reduced[col].astype('category')
        data_reduced[col_categorical] = data_reduced[col_categorical].apply(lambda x: x.cat.codes)
        data_reduced.head(5)

        # In contrast, model inference is the process of using a trained model to infer a result from live data.
        X = data_reduced.drop(['fraud'], axis=1)
        y = data_reduced['fraud']
        print("shape==============", [len(X.values), len(X.values[0])])
        print("X.values[0]==============", X.values[0], "=======",  list(X.values[0]))

        '''
            1. Using SMOTE (Synthetic Minority Oversampling Technique) for balancing the dataset. 
            2. Resulted counts show that now we have exact number of class instances (1 and 0).
        '''
        sm = SMOTE(random_state=42,k_neighbors=2)
        X_res, y_res = sm.fit_resample(X, y)
        y_res = pd.DataFrame(y_res)

        return X_res,y_res

print("check2")
a = A.remote()
print("check3")
print(ray.get(a.__call__.remote()))
print("check4")
print(ray.get(a.overspaling_smote.remote()))
print("check5")
ray.shutdown()

2023-08-18 15:11:16,796	INFO client_builder.py:237 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


check1
check2
check3
hello Avi, welcome to Ray !
check4
(A pid=804, ip=10.224.1.240) configuration_dtl :: ===== :: {'MINIO_HOST_URL': 'home.hpe-apps-ezaf.com:31900', 'MINIO_ACCESS_KEY': 'minioadmin', 'MINIO_SECRET_KEY': 'minioadmin', 'SOURCE_PATH': '/source/feed-2.csv', 'GENERATED_PATH': '/source/generated-data.csv', 'storage_uri': 's3://experiments/ray/pickels/logisticregression/model', 'protocol_version': 'v2', 'bucket_name': 'experiments', 'EZAF_ENV': 'hpe-staging-ezaf', 'token_url': 'https://keycloak.{0}.com/realms/UA/protocol/openid-connect/token'}
(A pid=804, ip=10.224.1.240) MINIO_CLIENT <minio.api.Minio object at 0x7fce510ad910>
(A pid=804, ip=10.224.1.240) shape============== [20, 7]
(A pid=804, ip=10.224.1.240) X.values[0]============== [ 0.   10.    3.    0.    1.    2.   10.59] ======= [0.0, 10.0, 3.0, 0.0, 1.0, 2.0, 10.59]
(    step  customer  age  gender  merchant  category      amount
0      0        10    3       0         1         2   10.590000
1      0         7    2